In [38]:
##########################
# Goal of this code is to save the mean state variables: u,v,omega,T,q,RH (3D) and PW (2D)
# 2024.3.27
# Mu-Ting Chien
######################################

In [39]:
import os
import sys
sys.path.append('/glade/work/muting/function/')
import KW_diagnostics as KW
import numpy as np
import matplotlib.pyplot as plt

# for function only
import glob
from tkinter import Tcl
from netCDF4 import Dataset


In [40]:
dir_out            = '/glade/work/muting/KW/'
CASENAME_LIST2     = list(['SST_AQP3_Qobs_27_-4K',\
                          'SST_AQP3_Qobs_27',\
                          'SST_AQP3_Qobs_27_4K'])
output_dir_all = dir_out+'output_data/All_3hr_10yr/'
CASENAME_SHORT_LIST = list(['-4K','CTL','4K'])
iyr_min = 2
iyr_max = 11
latmax  = 90

In [41]:
def load_3D_data_as_mean_state(CASENAME, CASENAME_SHORT, vname, iyr_min, iyr_max, latmax, nfile_skip):
    # vname is list(['vname1','vname2','vname3']), number of element in the list could be variable, up to 4 variables
    # iyr_min = yr to start (02-->year 3)
    # iyr_max = yr to end (04-->year 5)
    # latmax = 10 for the tropics
    #   For example, original unit of precip is kg/m^2/s. To convert to mm/day, we need to multiply by 1000*s2d.

    # Constants
    nmon = 12
    nt_s = 31
    ist   = 0
    nyr = iyr_max-iyr_min+1
    nt_m = nyr*nmon*8-nfile_skip # to calculate mean state
    nt = nt_m*nt_s
    nfile_skip_test = 0

    # Find how many variables
    vnum = np.size(vname)

    DIR_in = '/glade/derecho/scratch/muting/FROM_CHEYENNE/'
    file_dir = DIR_in+'work_output_pressure_coord_full/'+CASENAME+'/'
    ###################
    # load CESM 3-hourly output!
    for iyr in range(iyr_min,iyr_max+1):
        yr_str = "%04d" %(iyr+1)
        for imon in range(0,nmon):
            mon_str = "%02d" %(imon+1)
            for iday in range(0,8):
                day_str = "%02d" %(iday)
                file_in = file_dir+CASENAME_SHORT+'_'+yr_str+mon_str+day_str+'.nc'
                f = glob.glob(file_in)
                if len(f)==0:
                    nfile_skip_test = nfile_skip_test+1
                    continue
                data    = Dataset( file_in, "r", format="NETCDF4")
                if  iday == 0 and imon == 0 and iyr==iyr_min:
                    lon = data.variables['lon'][:]
                    nlon = np.size(lon)
                    lat_org = data.variables['lat'][:]
                    dmin = np.abs(lat_org+latmax)
                    dmax = np.abs(lat_org-latmax)
                    imin = np.argwhere(dmin==np.min(dmin)).squeeze()
                    imax = np.argwhere(dmax==np.min(dmax)).squeeze()
                    lat = lat_org[imin:imax+1]
                    nlat = np.size(lat)
                    #
                    plev = data.variables['plev'][:]
                    nlev = np.size(plev)
                    V1_zm = np.zeros([nt,nlat,nlev])

                temp= data.variables[vname[0]][:,imin:imax+1,:,:]
                nt_small = np.size(temp,0)

                if nt_small != nt_s:
                    print('Skip with this file, size of time not 31, YEAR:',yr_str,', MON:',mon_str,', FILE:',day_str)
                    continue

                V1_zm[ist:ist+nt_s,:,:]  = np.nanmean(temp,2)

                ist = ist+nt_s
    
    V1_zm_tm = np.nanmean(V1_zm,0)

    return V1_zm_tm, plev, lat

In [ ]:
for icase in range(0,3):
    
    CASENAME = CASENAME_LIST2[icase]+'_3h_20y'
    CASENAME_SHORT = CASENAME_SHORT_LIST[icase]+'_20y_3h_20y'

    # (0) Get nfile_skip
    nfile_skip = KW.find_nfile_skip(CASENAME, CASENAME_SHORT, iyr_min, iyr_max)

    # (1) Load u
    vname = list(['U'])
    um, plev, lat = load_3D_data_as_mean_state(CASENAME, CASENAME_SHORT, vname, iyr_min, iyr_max, latmax, nfile_skip)
    print('finish loading u')

    # Load T
    vname = list(['T'])
    Tm, plev, lat = load_3D_data_as_mean_state(CASENAME, CASENAME_SHORT, vname, iyr_min, iyr_max, latmax, nfile_skip)
    print('finish loading T')

    # Load omega
    vname = list(['OMEGA'])
    wm, plev, lat = load_3D_data_as_mean_state(CASENAME, CASENAME_SHORT, vname, iyr_min, iyr_max, latmax, nfile_skip)
    print('finish loading W')

    # Load v
    vname = list(['V'])
    vm, plev, lat = load_3D_data_as_mean_state(CASENAME, CASENAME_SHORT, vname, iyr_min, iyr_max, latmax, nfile_skip)
    print('finish loading V')

    # Load q
    vname = list(['Q'])
    qm, plev, lat = load_3D_data_as_mean_state(CASENAME, CASENAME_SHORT, vname, iyr_min, iyr_max, latmax, nfile_skip)
    print('finish loading Q')

    # Load rh
    vname = list(['RELHUM'])
    rhm, plev, lat = load_3D_data_as_mean_state(CASENAME, CASENAME_SHORT, vname, iyr_min, iyr_max, latmax, nfile_skip)
    print('finish loading RH')

    
    if icase == 0:
        nlev = np.size(plev)
        nlat = np.size(lat)
        um_all = np.empty([nlat, nlev, 3])
        Tm_all = np.empty([nlat, nlev, 3])
        vm_all = np.empty([nlat, nlev, 3])
        wm_all = np.empty([nlat, nlev, 3])
        qm_all = np.empty([nlat, nlev, 3])
        rhm_all = np.empty([nlat, nlev, 3])

    um_all[:,:,icase] = um
    Tm_all[:,:,icase] = Tm
    vm_all[:,:,icase] = vm
    wm_all[:,:,icase] = wm
    qm_all[:,:,icase] = qm
    rhm_all[:,:,icase] = rhm

    np.savez(output_dir_all+'mean_state_Tuvwq_all.npz', plev=plev, lat=lat, um=um_all, Tm=Tm_all, vm=vm_all, qm=qm_all, wm=wm_all, rhm=rhm_all)
    print('finish loading all variables')

In [47]:
print(np.min(um_all), np.max(um_all))

-13.790173693736016 47.995074787422446


In [48]:
print(np.min(Tm_all), np.max(Tm_all))

192.995684278998 301.8827329580976
